In [ ]:
import requests
import os
import json
from adalflow.core import Component, Generator
from adalflow.components.model_client import AnthropicAPIClient
from adalflow.core import DataClass, required_field
from adalflow.components.output_parsers.outputs import JsonOutputParser
from dataclasses import dataclass, field
import adalflow as adal
from dotenv import load_dotenv
load_dotenv()

: 

In [ ]:
class GenerationComponent:
    def __init__(self, model_client, model_kwargs, json_parser):
        self.generator = adal.Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            prompt_kwargs={},
            use_cache=True,
            output_processors=json_parser,
        )

    def call(self, task_desc_str, input_str, context_str=None, id=None):
        if context_str is None:
            output = self.generator.call(prompt_kwargs={"task_desc_str": task_desc_str, "input_str": input_str}, id=id)
        else:
            output = self.generator.call(prompt_kwargs={"task_desc_str": task_desc_str, "input_str": input_str, "context_str": context_str}, id=id)
        return output

In [3]:
@dataclass
class PatentOutline(DataClass):
    abstract: str = field(
        metadata={"desc": "Comprehensive summary of the product in around 250 words"}, default_factory=required_field()
    ) # required field
    detailed_description: list = field(
        metadata={"desc": "An outline formatted as a list of all of the functionality and nuances of the product"}, default_factory=required_field()
    ) # required field
    title: str = field(
        metadata={"desc": "A title for the proposed invention"}, default_factory=required_field()
    ) # required field
    search_query: list = field(
        metadata={"desc": "A list of 3 search queries for retrieving relevant information that could be used to flesh out the patent."}, default_factory=required_field()
    )

In [4]:
complete_parser = JsonOutputParser(data_class=PatentOutline)
complete_format_instructions = complete_parser.format_instructions()

In [5]:
model_client=AnthropicAPIClient(api_key=os.getenv("ANTHROPIC_API_KEY"))
model_kwargs={"model": "claude-3-5-sonnet-20240620", "max_tokens": 2048}
generator = GenerationComponent(model_client, model_kwargs, complete_parser)
result = generator.call("You are a helpful assistant providing comprehensive sections of patents based on inventions described in transcripts." + complete_format_instructions, "Here is some information about a blood collection device that is used to get blood sampling data")

In [7]:
result.data

{'abstract': 'This invention relates to an advanced blood collection device designed to revolutionize the process of obtaining blood sampling data. The device integrates cutting-edge technology with user-friendly features to enhance accuracy, efficiency, and patient comfort during blood collection procedures. It incorporates a minimally invasive needle system, real-time data processing capabilities, and seamless integration with electronic health records. The device is equipped with sensors to monitor blood flow and volume, ensuring precise sample collection. Additionally, it features a smart interface that guides healthcare professionals through the collection process, reducing the risk of errors. This innovative blood collection device aims to streamline laboratory workflows, improve diagnostic accuracy, and ultimately enhance patient care in various medical settings.',
 'detailed_description': ['Minimally invasive needle system for reduced patient discomfort',
  'Real-time data proc